In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd
import time
from functools import reduce
from sqlalchemy import create_engine

In [2]:
#initialize connection to spotify API

cid='66e8d1ba9950407db6d01d6c1bc6ecfd'
secret='c1f9f5ebaec249c0bee32fc1d127454b'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace = False

In [3]:
#sample from spotify-generated playlists
def getTrackIDs(user, playlist_id):
    track_ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        track_ids.append(track['id'])
    return track_ids

In [4]:
def getTrackFeatures(id):
    track_info = sp.track(id)
    features_info = sp.audio_features(id)

#track metadata
    name = track_info['name']
    album = track_info['album']['name']
    artist = track_info['album']['artists'][0]['name']
    release_date = track_info['album']['release_date']
    length = track_info['duration_ms']
    popularity = track_info['popularity']
    
#mostly spotify proprietary audio features
    danceability = features_info[0]['danceability']
    energy = features_info[0]['energy']
    key = features_info[0]['key']
    loudness = features_info[0]['loudness']
    mode = features_info[0]['mode']
    speechiness = features_info[0]['speechiness']
    acousticness = features_info[0]['acousticness']
    instrumentalness = features_info[0]['instrumentalness']
    liveness = features_info[0]['liveness']
    valence = features_info[0]['valence']
    tempo = features_info[0]['tempo']
    time_signature = features_info[0]['time_signature']

    track_data = [name, album, artist, release_date, length, popularity, danceability, energy, 
                  key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, 
                  valence, tempo, time_signature]
    
    return track_data

In [5]:
def get_dfs(track_ids):
    track_data_list = []
    for i in range(len(track_ids)):
        time.sleep(.1)
        try:
            track_data = getTrackFeatures(track_ids[i])
            track_data_list.append(track_data)
        except:
            pass

        df =  pd.DataFrame(track_data_list, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 
                                                                     'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                                                                     'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                                                                     'time_signature'])
    return df

In [6]:
#make list of playlists for each genre
hip_hop_list = ['spotify:playlist:37i9dQZF1DX0XUsuxWHRQd', 
               'spotify:playlist:37i9dQZF1DX2RxBh64BHjQ', 
               'spotify:playlist:37i9dQZF1DX6GwdWRQMQpq', 
               'spotify:playlist:37i9dQZF1DX9iGsUcr0Bpa', 
               'spotify:playlist:37i9dQZF1DWY4xHQp97fN6', 
               'spotify:playlist:37i9dQZF1DX9oh43oAzkyx', 
               'spotify:playlist:37i9dQZF1DX186v583rmzp', 
               'spotify:playlist:37i9dQZF1DX4SrOBCjlfVi', 
               'spotify:playlist:37i9dQZF1DX6OgmB2fwLGd', 
               'spotify:playlist:37i9dQZF1DWT5MrZnPU1zD', 
               'spotify:playlist:37i9dQZF1DX0HRj9P7NxeE', 
               'spotify:playlist:37i9dQZF1DWUFmyho2wkQU', 
               'spotify:playlist:37i9dQZF1DXaxIqwkEGFEh', 
               'spotify:playlist:37i9dQZF1DX6xZZEgC9Ubl', 
               'spotify:playlist:37i9dQZF1DX76t638V6CA8', 
               'spotify:playlist:37i9dQZF1DX0Tkc6ltcBfU', 
               'spotify:playlist:37i9dQZF1DWT6MhXz0jw61', 
               'spotify:playlist:37i9dQZF1DX9sQDbOMReFI']

jazz_list = ['spotify:playlist:37i9dQZF1DWTtzPKJEaTC4', 
            'spotify:playlist:37i9dQZF1DWW2c0C8Vb2IR', 
            'spotify:playlist:37i9dQZF1DWXIuW81skHVz', 
            'spotify:playlist:37i9dQZF1DX55dNU0PWnO5', 
            'spotify:playlist:37i9dQZF1DWTR4ZOXTfd9K', 
            'spotify:playlist:37i9dQZF1DXbITWG1ZJKYt', 
            'spotify:playlist:37i9dQZF1DWVqfgj8NZEp1', 
            'spotify:playlist:37i9dQZF1DX4wta20PHgwo',
            'spotify:playlist:37i9dQZF1DWYxwmBaMqxsl', 
            'spotify:playlist:37i9dQZF1DWUb0uBnlJuTi', 
            'spotify:playlist:37i9dQZF1DWY3X53lmPYk9', 
            'spotify:playlist:37i9dQZF1DX71VcjjnyaBQ', 
            'spotify:playlist:37i9dQZF1DX5YTAi6JhwZm', 
            'spotify:playlist:37i9dQZF1DX6G7arXBXa3A',
            'spotify:playlist:37i9dQZF1DWXSyfX6gqDNp', 
            'spotify:playlist:37i9dQZF1DX6KyCRJzZneq', 
            'spotify:playlist:37i9dQZF1DWZZfLKhEkflI']

rock_list = ['spotify:playlist:37i9dQZF1DXcF6B6QPhFDv', 
             'spotify:playlist:37i9dQZF1DWWJOmJ7nRx0C', 
             'spotify:playlist:37i9dQZF1DX82GYcclJ3Ug', 
             'spotify:playlist:37i9dQZF1DWXRqgorJj26U', 
             'spotify:playlist:37i9dQZF1DX1rVvRgjX59F', 
             'spotify:playlist:37i9dQZF1DX3oM43CtKnRV', 
             'spotify:playlist:37i9dQZF1DX1spT6G94GFC', 
             'spotify:playlist:37i9dQZF1DWWwzidNQX6jx', 
             'spotify:playlist:37i9dQZF1DX3YMp9n8fkNx', 
             'spotify:playlist:37i9dQZF1DX11ghcIxjcjE']

pop_list = ['spotify:playlist:37i9dQZF1DXcBWIGoYBM5M', 
           'spotify:playlist:37i9dQZF1DX0kbJZpiYdZl', 
           'spotify:playlist:37i9dQZF1DX2L0iB23Enbq', 
           'spotify:playlist:37i9dQZF1DWUa8ZRTfalHk', 
           'spotify:playlist:37i9dQZF1DWWvvyNmW9V9a', 
           'spotify:playlist:37i9dQZF1DXbYM3nMM0oPk', 
           'spotify:playlist:37i9dQZF1DX0b1hHYQtJjp', 
           'spotify:playlist:37i9dQZF1DWTwnEm1IYyoj', 
           'spotify:playlist:37i9dQZF1DWYs83FtTMQFw', 
           'spotify:playlist:37i9dQZF1DWWEcRhUVtL8n', 
           'spotify:playlist:37i9dQZF1DXcxvFzl58uP7', 
           'spotify:playlist:37i9dQZF1DX0s5kDXi1oC5']

classical_list = ['spotify:playlist:37i9dQZF1DWWEJlAGA9gs0', 
                  'spotify:playlist:37i9dQZF1DWV0gynK7G6pD',
                  'spotify:playlist:37i9dQZF1DWXtBjoO4Thyy', 
                  'spotify:playlist:37i9dQZF1DWVFeEut75IAL', 
                  'spotify:playlist:37i9dQZF1DXdLD1ufgw60J', 
                  'spotify:playlist:37i9dQZF1DWYkztttC1w38', 
                  'spotify:playlist:37i9dQZF1DX2XWJkYVfE4v', 
                  'spotify:playlist:37i9dQZF1DXddGd6mP5X2a', 
                  'spotify:playlist:37i9dQZF1DXah8e1pvF5oE', 
                  'spotify:playlist:37i9dQZF1DWWQxllNWdnsR', 
                  'spotify:playlist:37i9dQZF1DX0DHxkHcf1Tl']

country_list = ['spotify:playlist:37i9dQZF1DX1lVhptIYRda', 
               'spotify:playlist:37i9dQZF1DXdgnLr18vPvu', 
               'spotify:playlist:37i9dQZF1DWW7RgkOJG32Y', 
               'spotify:playlist:37i9dQZF1DX1KHLbhJkg7B', 
               'spotify:playlist:37i9dQZF1DWTkxQvqMy4WW', 
               'spotify:playlist:37i9dQZF1DWYvU2z6HruAo', 
               'spotify:playlist:37i9dQZF1DWVpjAJGB70vU', 
               'spotify:playlist:37i9dQZF1DXaJXCbmtHVHV', 
               'spotify:playlist:37i9dQZF1DXdxUH6sNtcDe', 
               'spotify:playlist:37i9dQZF1DX8WMG8VPSOJC', 
               'spotify:playlist:37i9dQZF1DWYnwbYQ5HnZU', 
               'spotify:playlist:37i9dQZF1DXaiEFNvQPZrM', 
               'spotify:playlist:37i9dQZF1DWXdiK4WAVRUW']

latin_list = ['spotify:playlist:37i9dQZF1DX10zKzsJ2jva', 
             'spotify:playlist:37i9dQZF1DWUoGbRYcteyC', 
             'spotify:playlist:37i9dQZF1DWY7IeIP1cdjF', 
             'spotify:playlist:37i9dQZF1DX1hVRardJ30X', 
             'spotify:playlist:37i9dQZF1DXbSwbJpH6lAw', 
             'spotify:playlist:37i9dQZF1DWZoF06RIo9el', 
             'spotify:playlist:37i9dQZF1DXbdrcAZnP3Cy', 
             'spotify:playlist:37i9dQZF1DX1QnNyJOBQBv', 
             'spotify:playlist:37i9dQZF1DX4OjfOteYnH8', 
             'spotify:playlist:37i9dQZF1DX7Qo2zphj7u3', 
             'spotify:playlist:37i9dQZF1DX4V5eXk4NKqu', 
             'spotify:playlist:37i9dQZF1DX4qKWGR9z0LI']

dance_electronic_list = ['spotify:playlist:37i9dQZF1DX4dyzvuaRJ0n', 
                        'spotify:playlist:37i9dQZF1DX8tZsk68tuDw', 
                        'spotify:playlist:37i9dQZF1DXa8NOEUWPn9W', 
                        'spotify:playlist:37i9dQZF1DXdURFimg6Blm', 
                        'spotify:playlist:37i9dQZF1DWSf2RDTDayIx', 
                        'spotify:playlist:37i9dQZF1DX0BcQWzuB7ZO', 
                        'spotify:playlist:37i9dQZF1DX0hvSv9Rf41p', 
                        'spotify:playlist:37i9dQZF1DXaXB8fQg7xif', 
                        'spotify:playlist:37i9dQZF1DX2TRYkJECvfC', 
                        'spotify:playlist:37i9dQZF1DX6GJXiuZRisr', 
                        'spotify:playlist:37i9dQZF1DX8CopunbDxgW']

In [7]:
engine = create_engine('postgresql://user:joan/paul@localhost:5432/project3')

In [8]:
#helper function for pulling all above together and creating posgres table

def get_data_to_sql(playlists, tablename):
    tracklist = []
    for playlist in playlists:
        try:
            a = getTrackIDs('Spotify', playlist)
            tracklist.append(a)
        except:
            pass

    #flatten list
    tracklist = reduce(lambda x,y: x+y, tracklist)

    #create dataframe of song attributes for each song in track list
    df = get_dfs(tracklist)
    
    #push df to sql
    df.to_sql(tablename, engine)

In [ ]:
get_data_to_sql(hip_hop_list, 'hiphop')
get_data_to_sql(jazz_list, 'jazz')
get_data_to_sql(rock_list, 'rock')
get_data_to_sql(pop_list, 'pop')
get_data_to_sql(classical_list, 'classical')

In [43]:
get_data_to_sql(country_list, 'country')
get_data_to_sql(latin_list, 'latin')
get_data_to_sql(dance_electronic_list, 'electronic')

In [9]:
get_data_to_sql(rock_list, 'rock')
get_data_to_sql(pop_list, 'pop')

In [10]:
query = 'SELECT * FROM rock;'
        
pd.read_sql(query, engine)

,index,name,album,artist,release_date,length,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0,Waiting On A War,Waiting On A War,Foo Fighters,2021-01-14,253840,69,0.530,0.759,7,-7.067,1,0.0351,0.009840,0.000000,0.3190,0.502,131.999,4
1,1,All My Favorite Songs,OK Human,Weezer,2021-01-29,202703,63,0.384,0.628,1,-8.018,0,0.0310,0.126000,0.000010,0.3140,0.394,161.438,4
2,2,Typhoons,Typhoons,Royal Blood,2021-01-21,236933,67,0.601,0.723,8,-4.364,0,0.0384,0.000347,0.001100,0.3460,0.581,114.991,4
3,3,And So It Went (feat. Tom Morello),And So It Went,The Pretty Reckless,2021-01-08,270214,57,0.471,0.900,8,-2.283,0,0.0832,0.014800,0.000003,0.0820,0.362,91.318,4
4,4,Take Me Away,Take Me Away,Ayron Jones,2020-06-19,246932,59,0.475,0.844,1,-4.448,0,0.0622,0.000120,0.111000,0.2210,0.554,156.136,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,805,Shadow of the Season,Sweet Oblivion,Screaming Trees,1992,273813,46,0.163,0.937,0,-5.502,1,0.0882,0.000046,0.002970,0.3720,0.413,184.482,4
806,806,Jezebel Woman,Return To Olympus,Malfunkshun,1995-06-25,271255,40,0.216,0.736,9,-7.309,0,0.0407,0.000689,0.000002,0.0821,0.246,131.013,4
807,807,Losing Skin,Four,Seaweed,1993-09-01,163066,0,0.209,0.951,0,-6.326,1,0.0609,0.000007,0.006570,0.0829,0.471,100.947,4
808,808,Swallow My Pride,Dry as a Bone / Rehab Doll,Green River,1988-01-07,179800,0,0.347,0.869,4,-11.481,1,0.0925,0.420000,0.000000,0.7600,0.194,127.509,4
